# How to find answers using jupyter?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

How to sanity check the environment?


In [1]:
var vm = require('vm');
var fs = require('fs');
// TODO: exports, require, module, __filename, __dirname
if (typeof importNotebook == 'undefined') {
    require('child_process').execSync('npm install JSONStream');
}
if (typeof commandRegistry == 'undefined') {
    var commandRegistry = {};
    commandRegistry['commandRegistry'] = commandRegistry;
}
$$.done('import ready');


How to import a code cells from jupyter?

How to import cells of different {types}?


In [2]:
var getCells = (notebook, types) => {
    if (typeof types == 'undefined') {
        var types = 'code';
    }
    if (typeof types === 'string') {
        types = [types];
    }
    var cellsToRun = [];
    var cells = fs.createReadStream(notebook)
        .pipe(require('JSONStream').parse('cells.*'));
    cells.on('data', (cell) => {
        if(types.indexOf(cell.cell_type) > -1) {
            cellsToRun[cellsToRun.length] = cell;
        }
    });
    return new Promise((resolve, reject) => {
        cells.on('error', e => reject(e, notebook));
        cells.on('close', () => resolve(cellsToRun, notebook))
    });
};
commandRegistry['getCells'] = getCells;
$$.done(getCells);



How to create VM promises out of the cell content?

In [3]:
// supported format How to find answers.ipynb[2]
var promiseContext = (scripts, notebook) => {
    var tempGlobal = Object.assign({}, global);
    tempGlobal = Object.assign(tempGlobal, commandRegistry);
    tempGlobal.$$ = Object.assign({}, $$);
    var context = vm.createContext(tempGlobal);
    return scripts.map((cell, i) => {
        var script = new vm.Script(`\n\n;\n\n` + cell + '\n\n;\n\n');
        return ((resolve, reject) => {
            var result;
            try {
                var tmp$$ = Object.assign(tempGlobal.$$, {
                    _isAsync: false,
                    async: () => tmp$$._isAsync = true,
                    done: (d) => resolve(d, i)
                });
                tempGlobal.$$ = tmp$$;
                result = script.runInContext(context);
                if (!tmp$$._isAsync) {
                    return resolve(result, i);
                }
            } catch (e) {
                return resolve(e, i);
            }
        });
    });
};
commandRegistry['promiseContext'] = promiseContext;
$$.async();
var cell = [
    'console.log("Sucessfully ran a script in it\'s own context!")'
];
var promise = promiseContext(cell, 'this one')
    .map((p) => new Promise(p))[0]
    .then(() => $$.done(promiseContext));


How to resolve a list of promises?

How to execute all the cells in a notebook?



In [4]:

var runAllPromises = (promises, notebook) => {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (new Promise(func)).then((r, i) => {
                return r
            }).then(Array.prototype.concat.bind(result));
        });
    },Promise.resolve([])).catch(e => console.log(e));
};
commandRegistry['runAllPromises'] = runAllPromises;
$$.async();
var cells = [
    '$$.done("promise 1")',
    '$$.done("promise 2")',
    '$$.done("Sucessfully ran a all promises!")'
];
var promises = promiseContext(cells, 'this one')
runAllPromises(promises, 'this one')
    .then((results) => {
        $$.done(runAllPromises);
        return results;
    });


How to import a {notebook}?

How to test the import feature?


In [11]:
if (typeof importedNotebooks == 'undefined') {
    var importedNotebooks = [];
}
if (typeof importedFunctions == 'undefined') {
    var importedFunctions = {};
}
if(typeof commandRegistry === 'undefined') {
    commandRegistry = {};
}
try {
commandRegistry['importedNotebooks'] = importedNotebooks;
var importNotebook = (notebook) => {
    if(typeof notebook === 'undefined') {
        return Promise.resolve();
    }
    if (importedNotebooks.indexOf(notebook) > -1) {
        console.log('Already imported ' + importedNotebooks.toString());
        return Promise.resolve(importedFunctions[notebook]);
    }
    return getCells(notebook, 'code').then((cells) => {
        var sources = cells.map(cell => cell.source.join('').trim());
        var promises = promiseContext(sources, notebook);
        console.log('started notebook ' + notebook + ' with ' + cells.length + ' cells');
        importedNotebooks[importedNotebooks.length] = notebook;
        return runAllPromises(promises, notebook);
    }).then((results) => {
        var result = {};
        results.forEach((r, i) => {
            if (typeof r === 'function') {
                console.log('found function ' + notebook+'['+i+']');
                result[notebook+'['+i+']'] = r;
                commandRegistry[notebook+'['+i+']'] = r;
            }
        });
        importedFunctions[notebook] = result;
        return result;
    }).catch(e => console.log(e));
}
commandRegistry['importNotebook'] = importNotebook;

// test to make sure it works
$$.async();
var imports = importNotebook('How to find answers.ipynb')
        .then((results) => {
            console.log(results);
            $$.done(importNotebook);
            return results;
        }).catch((e) => $$.done(e));
} catch (e) {$$.done(e)}


>TODO: MAKE THIS HEADER SMALLER?

How to provide this command as a service to other notebooks?

What to include to create inheritence in jupyter?

How to turn jupyter notebooks in to modules?



In [1]:
// Does this work everywhere?
var $c = console.log;
if (typeof notebook == 'undefined') {
    var notebook = 'How to find answers.ipynb';
}
if (typeof importNotebook == 'undefined') {
    var importNotebook = true;
    try {
        var m, co = '', vm = require('vm');
        var source = require('fs').readFileSync(notebook).toString();
        var re = new RegExp('\\s*"cell_type": "code",\\s*"source":\\s*\\[\\s*([\\s\\S]*?\\$\\$.done[\\s\\S]*?)\\s*\\]\\s*,\s*', 'ig');
        while (m = re.exec(source)) { co += (co == '' ? '' : ',\n' ) + m[1] + ',"\\n"'; }
        co = '\n\$$ = get$$();\n\n' + JSON.parse('['+co+']').join('') + '\n\n;';
        global.getGlobal = () => global;
        global.get$$ = () => {
            var temp$$ = Object.assign({}, global.$$);
            return Object.assign(temp$$, {
                async: () => {},
                done: (done) => {}
            });
        };
        var tmpGlobal = Object.assign({}, global);
        var context = vm.createContext(tmpGlobal);
        (new vm.Script(co)).runInContext(context);
        $$.async();
        tmpGlobal.imports.then((done) => {
            Object.assign(global, tmpGlobal.commandRegistry);
            $$.done('successfully imported notebooks');
        });
    } catch (e) {
        $$.done(e);
    }
}


Sucessfully ran a script in it's own context!
started notebook How to find answers.ipynb with 9 cells
Sucessfully ran a script in it's own context!
Already imported How to find answers.ipynb
undefined
found function How to find answers.ipynb[1]
found function How to find answers.ipynb[2]
found function How to find answers.ipynb[3]
found function How to find answers.ipynb[4]
{ 'How to find answers.ipynb[1]': [Function: getCells],
  'How to find answers.ipynb[2]': [Function: promiseContext],
  'How to find answers.ipynb[3]': [Function: runAllPromises],
  'How to find answers.ipynb[4]': [Function: importNotebook] }


'successfully imported notebooks'

started notebook How to test import.ipynb with 6 cells
started notebook How to test import.ipynb with 6 cells
Already imported How to find answers.ipynb,How to test import.ipynb,How to test import.ipynb
Already imported How to find answers.ipynb,How to test import.ipynb,How to test import.ipynb
Already imported How to find answers.ipynb,How to test import.ipynb,How to test import.ipynb
Already imported How to find answers.ipynb,How to test import.ipynb,How to test import.ipynb


These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?


In [1]:

$$.done('done importing importing')
